# Projeto Marinha do Brasil

## Laboratório de Processamento de Sinais - UFRJ

### Autor: Vinícius dos Santos Mello <viniciusdsmello@poli.ufrj.br>

In [1]:
import os
import sys
import pickle
import numpy as np
import time

from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K

import sklearn.metrics
from sklearn.externals import joblib
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt

from Functions import TrainParameters as trnparams
from Functions import TrainFunctions
from Functions.StackedAutoEncoders import StackedAutoEncoders
from Functions import FunctionsDataVisualization
from Functions.StatisticalAnalysis import KLDiv, EstPDF

import multiprocessing

init_time = time.time()

m_time = time.time()
print 'Time to import all libraries: '+str(m_time-init_time)+' seconds'

analysis_name = 'StackedAutoEncoder'

# Enviroment variables
data_path = os.getenv('OUTPUTDATAPATH')
results_path = os.getenv('PACKAGE_NAME')

# paths to export results
base_results_path = '%s/%s'%(results_path,analysis_name)
pict_results_path = '%s/pictures_files'%(base_results_path)
files_results_path = '%s/output_files'%(base_results_path)

# For multiprocessing purpose
num_processes = multiprocessing.cpu_count()

# Read data
m_time = time.time()

# Database caracteristics
database = '4classes'
n_pts_fft = 1024
decimation_rate = 3
spectrum_bins_left = 400
development_flag = False
development_events = 400

# Check if LofarData has created...
if not os.path.exists('%s/%s/lofar_data_file_fft_%i_decimation_%i_spectrum_left_%i.jbl'%
                      (data_path,database,n_pts_fft,decimation_rate,spectrum_bins_left)):
    print 'No Files in %s/%s\n'%(data_path,database)
else:
    #Read lofar data
    [data,trgt,class_labels] = joblib.load('%s/%s/lofar_data_file_fft_%i_decimation_%i_spectrum_left_%i.jbl'%
                                           (data_path,database,n_pts_fft,decimation_rate,spectrum_bins_left))


    m_time = time.time()-m_time
    print 'Time to read data file: '+str(m_time)+' seconds'

    # correct format
    all_data = data
    all_trgt = trgt

    # turn targets in sparse mode
    from keras.utils import np_utils
    trgt_sparse = np_utils.to_categorical(all_trgt.astype(int))

    # Process data
    # unbalanced data to balanced data with random data creation of small classes

    # Same number of events in each class
    qtd_events_biggest_class = 0
    biggest_class_label = ''

    for iclass, class_label in enumerate(class_labels):
        if sum(all_trgt==iclass) > qtd_events_biggest_class:
            qtd_events_biggest_class = sum(all_trgt==iclass)
            biggest_class_label = class_label
        print "Qtd event of %s is %i"%(class_label,sum(all_trgt==iclass))
    print "\nBiggest class is %s with %i events"%(biggest_class_label,qtd_events_biggest_class)

    balanced_data = {}
    balanced_trgt = {}

    from Functions import DataHandler as dh
    m_datahandler = dh.DataHandlerFunctions()

    for iclass, class_label in enumerate(class_labels):
        if development_flag:
            class_events = all_data[all_trgt==iclass,:]
            if len(balanced_data) == 0:
                balanced_data = class_events[0:development_events,:]
                balanced_trgt = (iclass)*np.ones(development_events)
            else:
                balanced_data = np.append(balanced_data,
                                          class_events[0:development_events,:],
                                          axis=0)
                balanced_trgt = np.append(balanced_trgt,(iclass)*np.ones(development_events))
        else:
            if len(balanced_data) == 0:
                class_events = all_data[all_trgt==iclass,:]
                balanced_data = m_datahandler.CreateEventsForClass(
                    class_events,qtd_events_biggest_class-(len(class_events)))
                balanced_trgt = (iclass)*np.ones(qtd_events_biggest_class)
            else:
                class_events = all_data[all_trgt==iclass,:]
                created_events = (m_datahandler.CreateEventsForClass(all_data[all_trgt==iclass,:],
                                                                     qtd_events_biggest_class-
                                                                     (len(class_events))))
                balanced_data = np.append(balanced_data,created_events,axis=0)
                balanced_trgt = np.append(balanced_trgt,
                                          (iclass)*np.ones(created_events.shape[0]),axis=0)

    all_data = balanced_data
    all_trgt = balanced_trgt

    # turn targets in sparse mode
    from keras.utils import np_utils
    trgt_sparse = np_utils.to_categorical(all_trgt.astype(int))

Using TensorFlow backend.


Time to import all libraries: 2.09808349609e-05 seconds
Time to read data file: 1.02709698677 seconds
Qtd event of 0 is 12939
Qtd event of 1 is 29352
Qtd event of 2 is 11510
Qtd event of 3 is 23760

Biggest class is 1 with 29352 events
DataHandler Class: CreateEventsForClass
Original Size: (12939, 400)
DataHandler Class: CreateEventsForClass
Original Size: (29352, 400)
DataHandler Class: CreateEventsForClass
Original Size: (11510, 400)
DataHandler Class: CreateEventsForClass
Original Size: (23760, 400)


# Carregar parâmetros de treinamento

In [13]:
# Load train parameters
analysis_str = 'StackedAutoEncoder'
model_prefix_str = 'RawData'

trn_params_folder='%s/%s/%s_trnparams.jbl'%(results_path,analysis_str,analysis_name)
if os.path.exists(trn_params_folder):
    os.remove(trn_params_folder)
if not os.path.exists(trn_params_folder):
    trn_params = trnparams.SAENoveltyDetectionTrnParams(n_inits=1,
                                                       hidden_activation='tanh', # others tanh, relu, sigmoid, linear 
                                                       output_activation='linear',
                                                       n_epochs=500,
                                                       patience=30,
                                                       batch_size=128,
                                                       verbose=2,
                                                       optmizerAlgorithm='Adam',
                                                       metrics=['accuracy'], #mean_squared_error
                                                       loss='mean_squared_error') #kullback_leibler_divergence
    trn_params.save(trn_params_folder)
else:
    trn_params = trnparams.NeuralClassificationTrnParams()
    trn_params.load(trn_params_folder)
    
# Choose how many fold to be used in Cross Validation
n_folds = 2
CVO = trnparams.ClassificationFolds(folder=results_path,
                                    n_folds=n_folds,
                                    trgt=all_trgt,
                                    dev=development_flag,
                                    verbose=True)
trn_params.printParams()
#print '\n'+trn_params.get_params_str()

File /home/vinicius.mello/Workspace/SonarAnalysis/Results/NoveltyDetection/2_folds_cross_validation.jbl exists
beta_1: 0.9
metrics: ['accuracy']
beta_2: 0.999
verbose: 2
loss: mean_squared_error
nesterov: True
epsilon: 1e-08
learning_rate: 0.001
n_inits: 1
batch_size: 128
n_epochs: 500
train_verbose: False
patience: 30
learning_decay: 1e-06
momentum: 0.3
optmizerAlgorithm: Adam
output_activation: linear
hidden_activation: tanh
norm: mapstd


# Treinamento do Stacked Auto Encoder

In [14]:
%time
SAE = StackedAutoEncoders(params           = trn_params,
                          development_flag = development_flag,
                          n_folds          = n_folds,
                          save_path        = results_path,
                          CVO              = CVO,
                          noveltyDetection = False)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.1 µs


In [ ]:
##### Train Process #####

# Choose layer to be trained
layer = 1

n_folds = len(CVO)

hidden_neurons = range(400,0,-50) + [2]
print hidden_neurons

regularizer = "dropout"
regularizer_param = 0.5

# Functions defined to be used by multiprocessing.Pool()
def trainNeuron(ineuron):
    for ifold in range(n_folds):
        SAE.trainLayer(data = all_data,
                        trgt = all_trgt,
                        ifold = ifold,
                        hidden_neurons = hidden_neurons + [ineuron],
                        layer = layer,
                        regularizer = regularizer,
                        regularizer_param = regularizer_param)

def trainFold(ifold):
    return  SAE.trainLayer(data = all_data,
                            trgt = all_trgt,
                            ifold = ifold,
                            hidden_neurons = hidden_neurons,
                            layer = layer,
                            regularizer = regularizer,
                            regularizer_param = regularizer_param)

start_time = time.time()

if K.backend() == 'theano':
    # Start Parallel processing
    p = multiprocessing.Pool(processes=num_processes)

    ####################### SAE LAYERS ############################
    # It is necessary to choose the layer to be trained

    # To train on multiple cores sweeping the number of folds
    folds = range(len(CVO))
    results = p.map(trainFold, folds)

    # To train multiple topologies sweeping the number of neurons
    # neurons_mat = range(0,400,50) (start,final,step)
    # results = p.map(trainNeuron, neurons_mat)

    p.close()
    p.join()
else: 
    for ifold in range(len(CVO)):
        result = trainFold(ifold)

end_time = time.time() - start_time
print "It took %.3f seconds to perform the training"%(end_time)

In [18]:
CVO

{0: [(array([14676, 14677, 14678, ..., 88053, 88054, 88055]),
   array([    0,     1,     2, ..., 73377, 73378, 73379])),
  (array([    0,     1,     2, ..., 73377, 73378, 73379]),
   array([14676, 14677, 14678, ..., 88053, 88054, 88055]))],
 1: [(array([14676, 14677, 14678, ..., 88053, 88054, 88055]),
   array([    0,     1,     2, ..., 73377, 73378, 73379])),
  (array([    0,     1,     2, ..., 73377, 73378, 73379]),
   array([14676, 14677, 14678, ..., 88053, 88054, 88055]))],
 2: [(array([14676, 14677, 14678, ..., 88053, 88054, 88055]),
   array([    0,     1,     2, ..., 73377, 73378, 73379])),
  (array([    0,     1,     2, ..., 73377, 73378, 73379]),
   array([14676, 14677, 14678, ..., 88053, 88054, 88055]))],
 3: [(array([14676, 14677, 14678, ..., 88053, 88054, 88055]),
   array([    0,     1,     2, ..., 73377, 73378, 73379])),
  (array([    0,     1,     2, ..., 73377, 73378, 73379]),
   array([14676, 14677, 14678, ..., 88053, 88054, 88055]))]}

In [8]:
n_folds = len(CVO)
for ifold in range(n_folds):
    train_id, test_id = CVO[ifold]
    
    # normalize classes
    if trn_params.params['norm'] == 'mapstd':
        scaler = preprocessing.StandardScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapstd_rob':
        scaler = preprocessing.RobustScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapminmax':
        scaler = preprocessing.MinMaxScaler().fit(all_data[train_id,:])
    norm_data[ifold] = scaler.transform(all_data)
    norm_data[ifold] = norm_data[ifold][test_id, :]
    
    models[ifold] = SAE.getModel(data=all_data, trgt=all_trgt,
                                 hidden_neurons=hidden_neurons,
                                 layer=layer, ifold=ifold)
    

ValueError: Found array with dim 3. StandardScaler expected <= 2.

# Estimação de PDF

In [ ]:
%time
# Reconstruction Analysis
%matplotlib inline 

# generate analysis data
save_path=results_path

current_analysis = 'pdf_estimation'
analysis_str = 'StackedAutoEncoder'
model_prefix_str = 'RawData'
verbose = False

# Choose layer 
layer = 1 

# Choose neurons topology
hidden_neurons = [400] #range(400,0,-50) + [2]

# Choose model
regularizer = ""
regularizer_param = 0.5

# Plot parameters
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.fontsize'] = 18
m_colors = ['b', 'r', 'g', 'y']

params_str = trn_params.get_params_str()

neurons_str = SAE.getNeuronsString(all_data, hidden_neurons=hidden_neurons[:layer])

models = {}
outputs = {}
norm_data = {}

n_folds = len(CVO)
for ifold in range(n_folds):
    train_id, test_id = CVO[ifold]
    
    # normalize classes
    if trn_params.params['norm'] == 'mapstd':
        scaler = preprocessing.StandardScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapstd_rob':
        scaler = preprocessing.RobustScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapminmax':
        scaler = preprocessing.MinMaxScaler().fit(all_data[train_id,:])
    norm_data[ifold] = scaler.transform(all_data)
    norm_data[ifold] = norm_data[ifold][test_id, :]
    
    models[ifold] = SAE.getModel(data=all_data, trgt=all_trgt,
                                 hidden_neurons=hidden_neurons,
                                 layer=layer, ifold=ifold)
    
    outputs[ifold] = models[ifold].predict(norm_data[ifold])

reconstructed_data = {}
klDivergenceFreq = {}

n_bins = 100

def getKlDiv(ifold):
    kl = np.zeros([all_data.shape[1]], dtype=object)
    for ifrequency in range(0,400):
        data = norm_data[ifold][:,ifrequency]
        reconstructed_data = outputs[ifold][:,ifrequency]
        m_bins = np.linspace(data.min(), data.max(), n_bins)
        kl[ifrequency] = KLDiv(data.reshape(-1,1), reconstructed_data.reshape(-1,1),
                               bins=m_bins, mode='kernel', kernel='epanechnikov',
                               kernel_bw=0.1, verbose=False)
        kl[ifrequency] = kl[ifrequency][0]
    return ifold, kl
# Start Parallel processing
p = multiprocessing.Pool(processes=num_processes)

folds = range(len(CVO))
klDivergenceFreq = p.map(getKlDiv, folds)

p.close()
p.join()
print 'Finished!'

In [ ]:
freqs = range(0,400)
for ifold in range(n_folds):
    fig, m_ax = plt.subplots(figsize=(10,10),nrows=1, ncols=1)
    m_ax.plot(freqs, klDivergenceFreq[ifold][1])
    m_ax.set_title('KL Divergence x Frequency - Layer %i - Fold %i'%(layer, ifold), fontweight='bold', fontsize=16)
    m_ax.set_xlabel('Frequency', fontsize=18)
    m_ax.set_ylabel('Kullback-Leibler Divergence', fontsize=18)

In [ ]:
# Neuron variation x KL Divergence
%matplotlib inline 

# generate analysis data
save_path=results_path

current_analysis = 'klDivergence'
analysis_str = 'StackedAutoEncoder'
model_prefix_str = 'RawData'

analysis_file_name='%s/%s/%s_%s_neuron_number_sweep.jbl'%(results_path,analysis_str,analysis_name,current_analysis)

verbose = True

# Choose layer 
layer = 1 

# Choose neurons topology
hidden_neurons = range(400,0,-50) + [2]

neurons_mat = [1, 100, 200, 300, 350, 400]

# Choose model
regularizer = ""
regularizer_param = 0.5

# Plot parameters
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.fontsize'] = 18
m_colors = ['b', 'r', 'g', 'y']

params_str = trn_params.get_params_str()

neurons_str = SAE.getNeuronsString(all_data, hidden_neurons=hidden_neurons[:layer])

n_folds = len(CVO)

klDivergence = np.zeros([n_folds,len(neurons_mat)], dtype=object)
klDivergenceFreq = {}

for ineuron in neurons_mat:
    
    neurons_str = SAE.getNeuronsString(all_data, hidden_neurons=hidden_neurons[:layer-1]+[ineuron])

    models = {}
    outputs = {}
    norm_data = {}
    if verbose: 
        print '[*] Layer: %i - Topology: %s'%(layer, neurons_str)
        
    reconstructed_data = {}
    
    n_bins = 100

    def getKlDiv(ifold):
        train_id, test_id = CVO[ifold]

        # normalize classes
        if trn_params.params['norm'] == 'mapstd':
            scaler = preprocessing.StandardScaler().fit(all_data[train_id,:])
        elif trn_params.params['norm'] == 'mapstd_rob':
            scaler = preprocessing.RobustScaler().fit(all_data[train_id,:])
        elif trn_params.params['norm'] == 'mapminmax':
            scaler = preprocessing.MinMaxScaler().fit(all_data[train_id,:])
        norm_data = scaler.transform(all_data)
        norm_data = norm_data[test_id, :]

        models[ifold] = SAE.getModel(data=all_data, trgt=all_trgt,
                                     hidden_neurons=hidden_neurons[:layer-1]+[ineuron],
                                     layer=layer, ifold=ifold)

        output = models[ifold].predict(norm_data)
        kl = np.zeros([all_data.shape[1]], dtype=object)
        for ifrequency in range(0,400):
            data = norm_data[:,ifrequency]
            reconstructed_data = output[:,ifrequency]
            m_bins = np.linspace(data.min(), data.max(), n_bins)
            kl[ifrequency] = KLDiv(data.reshape(-1,1), reconstructed_data.reshape(-1,1),
                                   bins=m_bins, mode='kernel', kernel='epanechnikov',
                                   kernel_bw=0.1, verbose=False)
            kl[ifrequency] = kl[ifrequency][0]
        return ifold, kl
    
    # Start Parallel processing
    p = multiprocessing.Pool(processes=num_processes)

    folds = range(len(CVO))
    if verbose:
        print '[*] Calculating KL Div for all frequencies'
    # Calculate the KL Div at all frequencies
    klDivergenceFreq[ineuron] = p.map(getKlDiv, folds)
    
    p.close()
    p.join()

# Plot results    
fig, m_ax = plt.subplots(figsize=(20,10),nrows=1, ncols=2)
for ifold in range(n_folds):
    # Sum the KL Div for all frequencies
    for index in range(len(neurons_mat)):
        klDivergence[ifold, index] = np.sum(klDivergenceFreq[neurons_mat[index]][ifold][1])
        
    m_ax[ifold].plot(neurons_mat, klDivergence[ifold,:], '-o')
    m_ax[ifold].set_title('KL Divergence x Neurons - %i Layer - %i Fold'%(layer,ifold+1), fontsize=16, fontweight='bold')
    m_ax[ifold].set_ylabel('Kullback-Leibler Divergence', fontsize=22)
    m_ax[ifold].set_xlabel('Neurons', fontsize=22)
    m_ax[ifold].grid()
plt.show()

In [ ]:
ineuron = 1
reconstructed_data = {}

neurons_str = SAE.getNeuronsString(all_data, hidden_neurons=hidden_neurons[:layer-1]+[ineuron])

models = {}
outputs = {}
norm_data = {}
if verbose: 
    print '[*] Layer: %i - Topology: %s'%(layer, neurons_str)
for ifold in range(n_folds):
    train_id, test_id = CVO[ifold]

    # normalize classes
    if trn_params.params['norm'] == 'mapstd':
        scaler = preprocessing.StandardScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapstd_rob':
        scaler = preprocessing.RobustScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapminmax':
        scaler = preprocessing.MinMaxScaler().fit(all_data[train_id,:])
    norm_data[ifold] = scaler.transform(all_data)
    norm_data[ifold] = norm_data[ifold][test_id, :]

    models[ifold] = SAE.getModel(data=all_data, trgt=all_trgt,
                                 hidden_neurons=hidden_neurons[:layer-1]+[ineuron],
                                 layer=layer, ifold=ifold)

    outputs[ifold] = models[ifold].predict(norm_data[ifold])

reconstructed_data = {}

n_bins = 100

def getKlDiv(ifold):
    kl = np.zeros([all_data.shape[1]], dtype=object)
    for ifrequency in range(0,400):
        data = norm_data[ifold][:,ifrequency]
        reconstructed_data = outputs[ifold][:,ifrequency]
        m_bins = np.linspace(data.min(), data.max(), n_bins)
        kl[ifrequency] = KLDiv(data.reshape(-1,1), reconstructed_data.reshape(-1,1),
                               bins=m_bins, mode='kernel', kernel='epanechnikov',
                               kernel_bw=0.1, verbose=False)
        kl[ifrequency] = kl[ifrequency]
    return ifold, kl
kl = {}
for ifold in range(n_folds):
    kl[ifold] = getKlDiv(ifold)

In [ ]:
kl[0][1]

# Análise de Correlação

In [ ]:
# Reconstruction Analysis
%matplotlib inline 

# generate analysis data
save_path=results_path

current_analysis = 'reconstruction'
analysis_str = 'StackedAutoEncoder'
model_prefix_str = 'RawData'
verbose = False

# Choose layer 
layer = 4 

# Choose neurons topology
hidden_neurons = range(400,0,-50) + [2]

# Choose model
regularizer = ""
regularizer_param = 0.5

# Plot parameters
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.fontsize'] = 18
m_colors = ['b', 'r', 'g', 'y']

params_str = trn_params.get_params_str()

neurons_str = SAE.getNeuronsString(all_data, hidden_neurons=hidden_neurons[:layer])

models = {}
mean = {}
indexes = {}
outputs = {}

n_folds = len(CVO)

for ifold in range(n_folds):
    train_id, test_id = CVO[ifold]
    
    # normalize classes
    if trn_params.params['norm'] == 'mapstd':
        scaler = preprocessing.StandardScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapstd_rob':
        scaler = preprocessing.RobustScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapminmax':
        scaler = preprocessing.MinMaxScaler().fit(all_data[train_id,:])
    norm_data = scaler.transform(all_data)
    norm_data = norm_data[test_id, :]
    
    if ifold == 0:
        diffSquared = np.zeros([len(CVO), norm_data.shape[0], norm_data.shape[1]])
        
    print 'Topology: %s - fold %i'%(neurons_str, ifold)
    
    models[ifold] = SAE.getModel(data=all_data, trgt=all_trgt,
                                 hidden_neurons=hidden_neurons,
                                 layer=layer, ifold=ifold)
    
    outputs[ifold] = models[ifold].predict(norm_data)
    
    diffSquared[ifold] = np.power((norm_data - outputs[ifold]), 2)
      
mean = np.mean(np.mean(diffSquared, axis=0), axis=0)
indexes = np.argsort(mean)[::-1]

print "Topology (%s)"%trn_params.get_params_str()

for ifold in range(len(CVO)):
    train_id, test_id = CVO[ifold]
    
    # normalize classes
    if trn_params.params['norm'] == 'mapstd':
        scaler = preprocessing.StandardScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapstd_rob':
        scaler = preprocessing.RobustScaler().fit(all_data[train_id,:])
    elif trn_params.params['norm'] == 'mapminmax':
        scaler = preprocessing.MinMaxScaler().fit(all_data[train_id,:])
    norm_data = scaler.transform(all_data)
    norm_data = norm_data[test_id, :]
    
    points = norm_data.shape[0]
    
    # Number of dimensions to analyse (even number is better!)
    num_dim = 4
    fig, m_ax = plt.subplots(figsize=(20,20),nrows=2, ncols=2)
    for choose_index in range(num_dim):  
        ax = plt.subplot(2,2,choose_index+1)
        
        # Plot classes
        ax.plot(norm_data[:,indexes[choose_index]][0],
                outputs[ifold][:,indexes[choose_index]][0],
                "b.", label='Test Data', markersize=20)
        ax.plot(norm_data[:,indexes[choose_index]][:points],
                outputs[ifold][:,indexes[choose_index]][:points],
                "b.")
        
        plt.tight_layout()
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.set_title('Input x Output - Dim %i'%(indexes[choose_index]),fontsize=18, fontweight='bold')
        ax.set_xlim(-2, 7)
        ax.set_ylim(-4, 8)
        handles, labels = ax.get_legend_handles_labels()
        # sort both labels and handles by labels
        labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
        plt.grid() 
        
        # Small plot
        rect = [0.065, 0.55, 0.35, 0.4]
        ax1 = FunctionsDataVisualization.add_subplot_axes(ax,rect)
        
        eq_reconstruction = np.power((norm_data[:,indexes[choose_index]] - outputs[ifold][:,indexes[choose_index]]), 2)
 
        mq_bins = np.linspace(np.min(eq_reconstruction), np.max(eq_reconstruction), 50)

        n, bins, patches = ax1.hist(eq_reconstruction, bins=mq_bins,
                                    fc="b",
                                    alpha=0.8, normed=0)   
        ax1.set_xlim(0, np.max(mq_bins))
        ax1.set_title("Squared Absolute Error",fontsize=14, fontweight='bold')
        ax1.grid() 
        
        # Small plot
        rect = [0.5, 0.05, 0.45, 0.4]
        ax2 = FunctionsDataVisualization.add_subplot_axes(ax,rect)
        
        e_reconstruction = (norm_data[:,indexes[choose_index]] - outputs[ifold][:,indexes[choose_index]])
      
        m_bins = np.linspace(np.min(e_reconstruction), np.max(e_reconstruction), 50) 
        
        n, bins, patches = ax2.hist(e_reconstruction,bins=m_bins,
                                    fc="b",
                                    alpha=0.8, normed=0)   
        ax2.grid()
        ax2.set_title("Absolute Error",fontsize=14, fontweight='bold')
        ax2.set_xlim(-np.max(mq_bins), np.max(mq_bins))
        
        ax.legend(handles, labels, ncol=1, loc='upper right')
        plt.legend()

        mse = metrics.mean_squared_error(norm_data[:,indexes[choose_index]], outputs[ifold][:,indexes[choose_index]])
        ax.text(2, 6, 'MSE: %f'%mse, style='normal',fontsize=26, color='blue',
        bbox={'alpha':0.0, 'pad':10})

        #Save the figure
        file_name = pict_results_path+'/'+current_analysis+'_%i_layer_%s_neurons_%i_fold_'%(layer,
                                                                                            neurons_str,
                                                                                            ifold)+trn_params.get_params_str()+'.pdf'
        plt.savefig(file_name)
    plt.show()

# Treinamento de Classificação

In [ ]:
# Train classifiers to their corresponding folds
def trainClassifierFold(ifold): 
    return SAE.trainClassifier(data=all_data,
                               trgt = all_trgt,
                               ifold = ifold,
                               hidden_neurons=hidden_neurons,
                               layer = 1)

# Train classifier sweeping the number of layer
def trainClassifierLayer(ilayer):
    for ifold in range(len(CVO)):
        SAE.trainClassifier(data=all_data,
                            trgt = all_trgt,
                            ifold = ifold,
                            hidden_neurons=hidden_neurons,
                            layer = ilayer)
start_time = time.time()

if K.backend() == 'theano':
    # Start Parallel processing
    p = multiprocessing.Pool(processes=num_processes)

    ####################### SAE LAYERS ############################
    # It is necessary to choose the layer to be trained

    # To train on multiple cores sweeping the number of folds
    folds = range(len(CVO))
    results = p.map(trainClassifierFold, folds)

    # To train multiple topologies sweeping the number of layers
    # neurons_mat = range(0,400,50) (start,final,step)
    # results = p.map(trainClassifierLayer, neurons_mat)

    p.close()
    p.join()
else: 
    for ifold in range(len(CVO)):
        result = trainClassifierFold(ifold)

end_time = time.time() - start_time
print "It took %.3f seconds to perform the training"%(end_time)

In [ ]:
classifier = {}
for ifold in range(len(CVO)):
    classifier[ifold] = SAE.loadClassifier(data = all_data, trgt = all_trgt, hidden_neurons=hidden_neurons, layer=1, ifold = ifold)

In [ ]:
for ifold in range(len(CVO)):
    train_id, test_id = CVO[ifold]
    output = classifier[ifold].predict(all_data[test_id, :])
    print sklearn.metrics.classification_report(trgt_sparse[test_id], np.round(output), target_names=class_labels.values())

In [ ]:
import numpy as np

def KL(a, b):
    a = np.asarray(a, dtype=np.float)
    b = np.asarray(b, dtype=np.float)

    return np.sum(np.where(a != 0, a * np.log(a / b), 0))

values1 = [1.346112,1.337432,1.246655]
values2 = [1.033836,1.082015,1.117323]

print KL(values1, values2)